## Data Processing and Visualization

In [1]:
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

/Users/gautham/Documents/Kaggle/store-sales-forecasting/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
train_df = pd.read_csv("./train.csv")
train_df = train_df.rename({"date":"ds", "sales":"y"}, axis=1)

oil_df = pd.read_csv("oil.csv")
oil_df = oil_df.rename({"date":"ds"}, axis=1)

stores_df = pd.read_csv("stores.csv")
stores_dict = stores_df.set_index("store_nbr").to_dict("index")
holidays_df = pd.read_csv("holidays_events.csv")
test_df = pd.read_csv("test.csv")
test_df = test_df.rename({"date":"ds"}, axis=1)

### Interpolate oil_df

In [3]:
blank_oil_df = pd.DataFrame({"ds":pd.date_range(train_df["ds"].min(), test_df["ds"].max()).astype("str")})
oil_df = blank_oil_df.merge(oil_df, how="left", on="ds")
oil_df["dcoilwtico"] = oil_df["dcoilwtico"].interpolate("nearest")
oil_df.iloc[0, 1] = 93.14

### Process Holidays DF

In [4]:
nth_df = holidays_df[holidays_df["transferred"] == False]
th_df = holidays_df[holidays_df["type"] == "Transfer"]
th_df["description"] = th_df["description"].str.removeprefix("Traslado ")
all_holidays_df = pd.concat([nth_df, th_df], axis=0)[["date", "locale_name", "description"]]
all_holidays_df = all_holidays_df.rename({"date":"ds", "description":"holiday"}, axis=1)
all_holidays_df["lower_window"] = 0
all_holidays_df["upper_window"] = 1

/var/folders/70/9p60jp9112d1hs3c2p3vr1th0000gn/T/ipykernel_48296/2455401216.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  th_df["description"] = th_df["description"].str.removeprefix("Traslado ")


### Add Additional Regressors

In [5]:
train_df = train_df.merge(oil_df, how="left", on="ds")
test_df = test_df.merge(oil_df, how="left", on="ds")

# info cols
train_df = train_df.merge(stores_df[["store_nbr", "cluster"]], how="left", on="store_nbr")
test_df = test_df.merge(stores_df[["store_nbr", "cluster"]], how="left", on="store_nbr")

In [6]:
days_counts = train_df.groupby(["store_nbr", "family"])["ds"].agg("count")

stores = pd.unique(train_df["store_nbr"])
families = pd.unique(train_df["family"])

promotion_counts = train_df[["id", "onpromotion"]].groupby("onpromotion").agg("count")
num_promotions = promotion_counts.shape[0]

states = pd.unique(stores_df["state"])
cities = pd.unique(stores_df["city"])
types = pd.unique(stores_df["type"])
cities_per_state = stores_df[["state", "city"]].drop_duplicates().groupby("state").agg("count")
stores_per_city = stores_df[["city", "store_nbr"]].drop_duplicates().groupby("city").agg("count")
stores_per_cluster = stores_df[["cluster", "store_nbr"]].drop_duplicates().groupby("cluster").agg("count")

subset as necessary for implementation and debugging

In [7]:
#train_df = train_df[train_df["store_nbr"] < 4]
#test_df = test_df[test_df["store_nbr"] < 4]

### Prophet: one model per store, family

In [8]:
def msle(preds_df):
    return np.mean((np.log(1 + preds_df["y"].values) - np.log(1 + preds_df["yhat"].values))**2)

def ssle(preds_df):
    return np.sum((np.log(1 + preds_df["y"].values) - np.log(1 + preds_df["yhat"].values))**2)

In [9]:
def fit_bottom_up(key_cols, train_df, stores_dict, all_holidays_df):
    def fit(x_df):
        store_nbrs = x_df["store_nbr"].drop_duplicates()
        states = [stores_dict[snbr]["state"] for snbr in store_nbrs]
        cities = [stores_dict[snbr]["city"] for snbr in store_nbrs]
        filter = (all_holidays_df["locale_name"] == "Ecuador")
        for s in states:
            filter = filter | (all_holidays_df["locale_name"] == s)
        for c in cities:
            filter = filter | (all_holidays_df["locale_name"] == c)
        h_df = all_holidays_df[filter]
        h_df = h_df[["ds", "holiday", "lower_window", "upper_window"]]

        x_df = x_df.groupby("ds").agg({"y":"sum", "onpromotion":"sum", "dcoilwtico":"first"}).reset_index()

        model = Prophet(uncertainty_samples=0, holidays=h_df)
        model.add_regressor("onpromotion")
        model.add_regressor("dcoilwtico")

        model.fit(x_df)
        return model
    
    return train_df.groupby(key_cols).apply(fit).reset_index()

def predict_bottom_up(test_df, models_df):
    key_cols = models_df.columns[:-1].to_list()
    def predict(x_df):
        filter = pd.Series(models_df.shape[0] * [True])
        for k in key_cols:
            v = x_df[k].iloc[0]
            filter = filter & (models_df[k] == v)
        model = models_df[filter].iloc[0, -1]

        x_df = x_df.groupby("ds").agg({"onpromotion":"sum", "dcoilwtico":"first"}).reset_index()

        return model.predict(x_df)

    return test_df.groupby(key_cols).apply(predict).reset_index()

def all_cross_validation(key_cols, train_df, stores_dict, all_holidays_df):
    def cv(x_df):
        store_nbrs = x_df["store_nbr"].drop_duplicates()
        states = [stores_dict[snbr]["state"] for snbr in store_nbrs]
        cities = [stores_dict[snbr]["city"] for snbr in store_nbrs]
        filter = (all_holidays_df["locale_name"] == "Ecuador")
        for s in states:
            filter = filter | (all_holidays_df["locale_name"] == s)
        for c in cities:
            filter = filter | (all_holidays_df["locale_name"] == c)
        h_df = all_holidays_df[filter]
        h_df = h_df[["ds", "holiday", "lower_window", "upper_window"]]

        x_df = x_df.groupby("ds").agg({"y":"sum", "onpromotion":"sum", "dcoilwtico":"first"}).reset_index()


        model = Prophet(uncertainty_samples=0, holidays=h_df)
        model.add_regressor("onpromotion")
        model.add_regressor("dcoilwtico")
        model.fit(x_df)
        cv_df = cross_validation(model, initial='1460 days', period='56 days', horizon='16 days')
        cv_df["yhat"] = cv_df["yhat"].clip(lower=0)
        return cv_df.groupby("cutoff").apply(msle).reset_index()

    msles_df = train_df.groupby(key_cols).apply(cv).reset_index().rename({0:"msle"}, axis=1)
    return np.mean(np.sqrt(msles_df.groupby("cutoff")["msle"].agg("mean").values))


## Down-Aggregation

In [10]:
def get_proportions(key_cols, support_cols, x_df):
    x_df = x_df.groupby(key_cols + support_cols + ["ds"]).agg({"y":"sum", "onpromotion":"sum", "dcoilwtico":"first"}).reset_index()
    agg_x_df = x_df.groupby(key_cols + ["ds"]).agg({"y":"sum"}).reset_index().rename({"y":"agg_y"}, axis=1)
    x_df = x_df.merge(agg_x_df, how="left", on=key_cols + ["ds"])
    x_df["prop"] = x_df.loc[:, ["y"]].where(x_df["agg_y"] <= 0, x_df["y"] / x_df["agg_y"], axis=0)
    return x_df.drop(["y", "agg_y"], axis=1)

def year_diff(key_cols, support_cols, prop_df):
    other_df = prop_df.copy()
    other_df["ds"] = (pd.to_datetime(other_df["ds"]) + DateOffset(years=1)).dt.strftime("%Y-%m-%d")
    other_df = other_df.groupby(key_cols + support_cols + ["ds"]).agg("first").reset_index()
    other_df = other_df.rename({"prop":"prev_prop"}, axis=1)[key_cols + support_cols + ["ds", "prev_prop"]]
    leap_yrs_df = other_df[other_df["ds"] == "2016-02-28"]
    leap_yrs_df["ds"] = "2016-02-29"
    other_df = pd.concat([other_df, leap_yrs_df]).sort_values(key_cols + support_cols + ["ds"]).reset_index(drop=True)
    dprop_df = prop_df.merge(other_df, how="inner", on=key_cols + support_cols + ["ds"])
    dprop_df["prop"] = dprop_df["prop"] - dprop_df["prev_prop"]
    dprop_df = dprop_df[key_cols + support_cols + ["ds", "prop", "onpromotion", "dcoilwtico"]]
    return dprop_df, other_df

def year_inv_diff(key_cols, support_cols, prop_hat_df, other_df):
    prop_hat_df = prop_hat_df.merge(other_df, how="inner", on=key_cols + support_cols + ["ds"])
    prop_hat_df["prop_hat"] = prop_hat_df["prop_hat"] + prop_hat_df["prev_prop"]
    prop_hat_df = prop_hat_df[key_cols + support_cols + ["ds", "prop_hat"]]
    return prop_hat_df

class VARModel():
    def __init__(self, lag, support_cols):
        self.lag = lag
        self.support_cols = support_cols
    
    def fit(self, x_df, lmbda):
        px_df = x_df.pivot(columns=self.support_cols, index="ds", values="prop").sort_index()
        oil_df = x_df[["ds", "dcoilwtico"]].drop_duplicates().sort_values("ds")
        # need to use loop to build design matrix
        design_cols = []
        for l in range(self.lag):
            dc = px_df.iloc[l:(px_df.shape[0] - self.lag + l), :].values.flatten()
            design_cols.append(dc)
        design_cols.append(np.repeat(oil_df["dcoilwtico"].values[(self.lag):], px_df.shape[1]))
        X = np.stack(design_cols, axis=1)
        y = px_df.iloc[self.lag:, :].values.flatten()
        self.beta = lin_reg(X, y, lmbda)

        self.px = px_df.values[-self.lag:, :].T
        self.d = datetime.strptime(px_df.index[-1], "%Y-%m-%d").date() + timedelta(days=1)
        self.support = px_df.columns
    
    def predict(self, test_oil_df, days):
        test_oil_df = test_oil_df[["ds", "dcoilwtico"]].drop_duplicates().set_index("ds").sort_index()
        ox = np.full((self.px.shape[0], 1), test_oil_df.loc[self.d.strftime("%Y-%m-%d"), "dcoilwtico"])
        bx = np.ones((self.px.shape[0], 1))
        x = np.concatenate([bx, self.px, ox], axis=1)
        d0 = self.d
        out = []
        for i in range(days):
            if i > 0:
                self.px = np.concatenate([self.px[:, 1:], y[:, np.newaxis]], axis=1)
                self.d = self.d + timedelta(days=1)
                ox = np.full((self.px.shape[0], 1), test_oil_df.loc[self.d.strftime("%Y-%m-%d"), "dcoilwtico"])
                bx = np.ones((self.px.shape[0], 1))
                x = np.concatenate([bx, self.px, ox], axis=1)
                
            y = x @ self.beta
            out.append(y)
        ds = pd.date_range(start=d0.strftime("%Y-%m-%d"), periods=days, freq="D", inclusive="left").repeat(self.px.shape[0]).strftime("%Y-%m-%d")
        pdf_dict = {"ds":ds}
        if len(self.support_cols) == 1:
            pdf_dict[self.support_cols[0]] = np.tile(self.support.to_numpy(), (days, ))
        else:
            for j, sc in enumerate(self.support_cols):
                pdf_dict[sc] = np.tile(np.array([self.support[i][j] for i in range(len(self.support))]), (days, ))
            pass
        pdf_dict["prop_hat"] = np.concatenate(out)
        preds_df = pd.DataFrame(pdf_dict)
        return preds_df


def lin_reg(X, y, lmbda):
    X = np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)
    beta = np.linalg.solve(X.T @ X + lmbda * np.eye(X.shape[1]), X.T @ y)
    return beta

In [18]:
def fit_VAR(key_cols, support_cols, train_df):
    prop_df = get_proportions(key_cols, support_cols, train_df)
    prop_df, prev_prop_df = year_diff(key_cols, support_cols, prop_df)

    def fit(x_df):
        model = VARModel(lag=6, support_cols=support_cols)
        model.fit(x_df, 0)
        return model
    return prop_df.groupby(key_cols).apply(fit).reset_index(), prev_prop_df


def predict_VAR(key_cols, support_cols, test_df, var_models_df, prev_prop_df):
    def predict(x_df):
        days = x_df["ds"].drop_duplicates().shape[0]
        filter = pd.Series(var_models_df.shape[0] * [True])
        for k in key_cols:
            v = x_df[k].iloc[0]
            filter = filter & (var_models_df[k] == v)
        model = var_models_df[filter].iloc[0, -1]
        return model.predict(test_df, days)

    prop_hat_df = test_df.groupby(key_cols).apply(predict).reset_index()
    return year_inv_diff(key_cols, support_cols, prop_hat_df, prev_prop_df)

def top_down_cv(key_cols, support_cols, train_df, stores_dict, all_holidays_df, train_days=365*3, val_days=16, interval=64):
    d0 = datetime.strptime(train_df["ds"].iloc[0], "%Y-%m-%d").date()
    d_max = datetime.strptime(train_df["ds"].iloc[-1], "%Y-%m-%d").date()

    def cv(x_df):
        # x_df has all same key_cols but not grouped yet, contains y, dprop, and prev_prop

        # retrieve h_df
        store_nbrs = x_df["store_nbr"].drop_duplicates()
        states = [stores_dict[snbr]["state"] for snbr in store_nbrs]
        cities = [stores_dict[snbr]["city"] for snbr in store_nbrs]
        filter = (all_holidays_df["locale_name"] == "Ecuador")
        for s in states:
            filter = filter | (all_holidays_df["locale_name"] == s)
        for c in cities:
            filter = filter | (all_holidays_df["locale_name"] == c)
        h_df = all_holidays_df[filter]
        h_df = h_df[["ds", "holiday", "lower_window", "upper_window"]]

        # aggregate x_df on key and support cols
        x_df = x_df.groupby(key_cols + support_cols + ["ds"]).agg({"y":"sum", "onpromotion":"sum", "dcoilwtico":"first"}).reset_index().sort_values(key_cols + support_cols + ["ds"])
        # get proportions and transform
        x_prop_df = get_proportions(key_cols, support_cols, x_df)
        x_prop_df, x_prev_prop_df = year_diff(key_cols, support_cols, x_prop_df)
        x_prop_df = x_prop_df.sort_values(key_cols + support_cols + ["ds"])
        x_prev_prop_df = x_prev_prop_df.sort_values(key_cols + support_cols + ["ds"])

        # internal loop for each fold
        # for loop
        d = d0
        errs = []
        while d + timedelta(days=train_days) + timedelta(days=val_days - 1) <= d_max:
            # calculate train dates and test dates
            d1 = d.strftime("%Y-%m-%d")
            d2 = (d + timedelta(days=train_days)).strftime("%Y-%m-%d")
            d3 = (d + timedelta(days=train_days + val_days)).strftime("%Y-%m-%d")

            # retrieve agg train ys for prophet training
            x_agg_train_df = x_df[(x_df["ds"] >= d1) & (x_df["ds"] < d2)].groupby(key_cols + ["ds"]).agg({"y":"sum", "onpromotion":"sum", "dcoilwtico":"first"}).reset_index()
            # fit model
            model = Prophet(uncertainty_samples=0, holidays=h_df)
            model.add_regressor("onpromotion")
            model.add_regressor("dcoilwtico")
            model.fit(x_agg_train_df)
            # retrieve agg val df for propher prediction
            x_agg_val_df = x_df[(x_df["ds"] >= d2) & (x_df["ds"] < d3)].groupby(key_cols + ["ds"]).agg({"onpromotion":"sum", "dcoilwtico":"first"}).reset_index()
            # predict
            x_pred_df = model.predict(x_agg_val_df)[["ds", "yhat"]]
            x_pred_df["ds"] = np.datetime_as_string(x_pred_df["ds"].to_numpy(), unit='D')

            # retrieve train props for var training
            x_prop_train_df = x_prop_df[(x_prop_df["ds"] >= d1) & (x_prop_df["ds"] < d2)]
            # fit var model and predict
            var_model = VARModel(lag=8, support_cols=support_cols)
            var_model.fit(x_prop_train_df, 1e-1)
            x_prop_hat_df = var_model.predict(x_prop_df[(x_prop_df["ds"] >= d2) & (x_prop_df["ds"] < d3)], val_days)
            # transform var predictions using prev_props
            x_prop_hat_df = year_inv_diff([], support_cols, x_prop_hat_df, x_prev_prop_df)

            # merge predictions and multiply to get final predictions
            x_pred_df = x_prop_hat_df.merge(x_pred_df, on="ds", how="left")
            x_pred_df["yhat"] = (x_pred_df["yhat"] * x_pred_df["prop_hat"])
            x_pred_df = x_pred_df.drop("prop_hat", axis=1)
            x_pred_df["yhat"] = x_pred_df["yhat"].clip(lower=0)

            true_df = x_df[(x_df["ds"] >= d2) & (x_df["ds"] < d3)].drop(key_cols + ["onpromotion", "dcoilwtico"], axis=1)
            x_pred_df = x_pred_df.merge(true_df, how="left", on=support_cols + ["ds"])

            err = msle(x_pred_df)
            errs.append(err)

            # calculate rmsle

            # increment date
            d += timedelta(days=interval)
        return np.mean(np.array(errs))

        # merge prophet predictions and var predictions, then multiply to retrieve final prediction
    return train_df.groupby(key_cols).apply(cv)

In [12]:
key_cols = ["cluster"]
support_cols = ["family"]

In [13]:
# result_df = all_cross_validation(key_cols, train_df, stores_dict, all_holidays_df)

In [14]:
# cluster = 4
# prop_df = get_proportions(key_cols, support_cols, train_df)
# prop_df, prev_prop_df = year_diff(key_cols, support_cols, prop_df)
# x_df = prop_df[prop_df["cluster"] == cluster]

# model = VARModel(lag=6, support_cols=support_cols)
# model.fit(x_df, 0)
# prop_hat_df = model.predict(test_df, 16)
# prop_hat_df["cluster"] = cluster
# year_inv_diff(key_cols, support_cols, prop_hat_df, prev_prop_df)

In [15]:
# var_models_df, prev_prop_df = fit_VAR(key_cols, support_cols, train_df)
# predict_VAR(key_cols, support_cols, test_df, var_models_df, prev_prop_df)

In [19]:
result_df = top_down_cv(key_cols, support_cols, train_df, stores_dict, all_holidays_df)

22:43:24 - cmdstanpy - INFO - Chain [1] start processing
22:43:25 - cmdstanpy - INFO - Chain [1] done processing
22:43:25 - cmdstanpy - INFO - Chain [1] start processing
22:43:26 - cmdstanpy - INFO - Chain [1] done processing
22:43:27 - cmdstanpy - INFO - Chain [1] start processing
22:43:27 - cmdstanpy - INFO - Chain [1] done processing
22:43:28 - cmdstanpy - INFO - Chain [1] start processing
22:43:29 - cmdstanpy - INFO - Chain [1] done processing
22:43:29 - cmdstanpy - INFO - Chain [1] start processing
22:43:30 - cmdstanpy - INFO - Chain [1] done processing
22:43:31 - cmdstanpy - INFO - Chain [1] start processing
22:43:31 - cmdstanpy - INFO - Chain [1] done processing
22:43:32 - cmdstanpy - INFO - Chain [1] start processing
22:43:33 - cmdstanpy - INFO - Chain [1] done processing
22:43:33 - cmdstanpy - INFO - Chain [1] start processing
22:43:34 - cmdstanpy - INFO - Chain [1] done processing
22:43:35 - cmdstanpy - INFO - Chain [1] start processing
22:43:35 - cmdstanpy - INFO - Chain [1]

In [20]:
result_df

cluster
1     0.771655
2     0.797598
3     0.769825
4     0.607174
5     0.985876
6     0.747840
7     1.307439
8     1.025360
9     0.637526
10    0.829874
11    0.973251
12    0.717449
13    0.806018
14    1.109758
15    0.863875
16    1.374965
17    0.849184
dtype: float64